In [1]:
import sys
from pathlib import Path
import torch
import pandas as pd
from chronos import Chronos2Pipeline

REPO_ROOT = Path.cwd().parent.parent
BACKEND_DIR = REPO_ROOT / "backend"
sys.path.insert(0, str(BACKEND_DIR))
sys.path.insert(0, str(Path.cwd()))

from _pool_common import (
    load_pool_data,
    compute_metrics,
    metrics_to_parquet,
    TEST_SIZE,
    MIN_CONTEXT_CHRONOS,
    ARTIFACTS_DIR,
)

MODEL_ID = "amazon/chronos-2"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

c:\capstone_project_unfc\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(f"Loading {MODEL_ID} on {DEVICE}...")
pipeline = Chronos2Pipeline.from_pretrained(
    MODEL_ID,
    device_map=DEVICE,
    torch_dtype=torch.float32,
)

def chronos_predict_1step(train_series: pd.Series) -> float:
    target_values = train_series.values.flatten()
    timestamps = train_series.index
    input_df = pd.DataFrame({
        "item_id": ["x"] * len(target_values),
        "timestamp": timestamps,
        "target": target_values,
    })
    forecast_df = pipeline.predict_df(
        input_df,
        prediction_length=1,
        quantile_levels=[0.5],
        id_column="item_id",
        timestamp_column="timestamp",
        target="target",
    )
    return float(forecast_df["0.5"].iloc[0])

def backtest_one_step_chronos(prices_full: pd.Series, test_size: int):
    preds = []
    split_idx = len(prices_full) - test_size
    for i in range(split_idx, len(prices_full)):
        train = prices_full.iloc[:i]
        if len(train) < MIN_CONTEXT_CHRONOS:
            continue
        ts = prices_full.index[i]
        val = prices_full.iloc[i]
        y_true = float(val.item() if hasattr(val, "item") else val)
        try:
            y_pred = chronos_predict_1step(train)
            preds.append({"timestamp": ts, "y_true": y_true, "y_pred": y_pred})
        except Exception as e:
            print(f"Skip {ts}: {e}")
            continue
    return pd.DataFrame(preds)

Loading amazon/chronos-2 on cpu...


`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!


In [4]:
stacked = load_pool_data()
print(stacked.groupby("symbol").size())
stacked.head(10)

symbol
AAPL       262
BTC-USD    262
ETH-USD    262
MSFT       262
NVDA       262
QQQ        262
SPY        262
dtype: int64


,timestamp,symbol,close
0,2021-02-22,AAPL,121.260002
1,2021-03-01,AAPL,121.419998
2,2021-03-08,AAPL,121.029999
3,2021-03-15,AAPL,119.989998
4,2021-03-22,AAPL,121.209999
5,2021-03-29,AAPL,123.000000
6,2021-04-05,AAPL,133.000000
7,2021-04-12,AAPL,134.160004
8,2021-04-19,AAPL,134.320007
9,2021-04-26,AAPL,131.460007


In [5]:
model_name = "chronos"
all_preds = []
for sym, grp in stacked.groupby("symbol"):
    prices = grp.set_index("timestamp")["close"].astype(float).dropna().sort_index()
    if len(prices) < TEST_SIZE + MIN_CONTEXT_CHRONOS:
        continue
    pred = backtest_one_step_chronos(prices, TEST_SIZE)
    pred["symbol"] = sym
    all_preds.append(pred)

pred_chronos = pd.concat(all_preds, ignore_index=True)
print(pred_chronos.groupby("symbol").size())
pred_chronos.head()

c:\capstone_project_unfc\env\Lib\site-packages\chronos\chronos2\dataset.py:89: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:219.)
  task_target = torch.from_numpy(task_target)


symbol
AAPL       30
BTC-USD    30
ETH-USD    30
MSFT       30
NVDA       30
QQQ        30
SPY        30
dtype: int64


,timestamp,y_true,y_pred,symbol
0,2025-08-04,229.350006,202.887268,AAPL
1,2025-08-11,231.589996,225.876114,AAPL
2,2025-08-18,227.759995,229.483261,AAPL
3,2025-08-25,232.139999,226.869156,AAPL
4,2025-09-01,239.690002,231.294067,AAPL


In [6]:
metrics_rows = []
for sym in pred_chronos["symbol"].unique():
    sub = pred_chronos[pred_chronos["symbol"] == sym]
    m = compute_metrics(sub)
    metrics_rows.append({"model": model_name, "symbol": sym, **m})
m_overall = compute_metrics(pred_chronos)
metrics_rows.append({"model": model_name, "symbol": "overall", **m_overall})

metrics_df = pd.DataFrame(metrics_rows)
print(metrics_df.to_string())
metrics_to_parquet(metrics_rows, ARTIFACTS_DIR / "metrics_chronos_pool.parquet")
print("Saved:", ARTIFACTS_DIR / "metrics_chronos_pool.parquet")

     model   symbol          MAE         RMSE    MAPE_%
0  chronos     AAPL     7.969783     9.893981  3.132164
1  chronos  BTC-USD  4448.797135  5421.597027  4.565317
2  chronos  ETH-USD   248.480416   322.026208  7.174112
3  chronos     MSFT    11.335902    15.110289  2.430277
4  chronos     NVDA     5.769599     6.958983  3.144081
5  chronos      QQQ     8.972439    10.923290  1.486533
6  chronos      SPY     6.518620     8.267180  0.973254
7  chronos  overall   676.834842  2052.802179  3.272248
Saved: C:\capstone_project_unfc\model\experiments-pool\artifacts\metrics_chronos_pool.parquet
